# Simulación de Eventos Discretos - TP1

TODO: explicar modelo conceptual

Hace falta compilar el simulador con los atómicos nuevos:

In [ ]:
%%bash 

cd src/

make

## Atómico Producto

In [ ]:
%%bash

cat model/test_product.ma

In [ ]:
%%bash

cat model/test_product.ev

In [ ]:
%%bash

src/bin/cd++ -m model/test_product.ma \
             -t 10:00:00:000 \
             -l test_product.log \
             -o test_product.out \
             -e model/test_product.ev

In [ ]:
%%bash

cat model.out

## Atomico Mercado

In [ ]:
%%bash

cat model/test_market.ma

In [ ]:
%%bash

cat model/test_market.ev

In [ ]:
%%bash

src/bin/cd++ -m model/test_market.ma \
             -t 05:00:00:000 \
             -l test_market.log \
             -o test_market.out \

In [ ]:
%%bash

cat test_market.out

In [ ]:
%%bash

cat test_market.log01

## Atómico Pais

In [ ]:
%%bash

cat model/test_country.ma

In [ ]:
%%bash

cat model/test_country.ev

In [ ]:
%%bash

src/bin/cd++ -m model/test_country.ma \
             -t 05:00:00:000 \
             -l test_country.log \
             -o test_country.out \
             -e model/test_country.ev

In [ ]:
%%bash

cat test_country.out

In [ ]:
%%bash

src/bin/cd++ -m model/simulation.ma \
             -t 05:00:00:000 \
             -l simulation.log \
             -o simulation.out

## Modelo Completo

In [ ]:
%%bash

cd model

python3 generate_ma.py

In [ ]:
%%bash

mkdir -p logs
src/bin/cd++ -m model/simulation.ma \
             -t 10:00:00:000 \
             -l logs/simulation.log \
             -o simulation.out

In [ ]:
%%bash
head simulation.out

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# definimos los path a los archivos de interés
OUT_FILENAME = "simulation.out"

# definimos los nombres de las columnas en los dataframes de pandas
TIME_COL = 'time'
PORT_COL = 'port'
VALUE_COL = 'value'
# MESSAGE_TYPE_COL = 'message_type'
# MODEL_ORIGIN_COL = 'model_origin'
# MODEL_DEST_COL = 'model_dest'

# parsea cada fila del dataframe
def parse_value(value: str):
    if '?' in value: return None
    is_list = value.strip().startswith("[") and value.strip().endswith("]")
    if is_list:
        return tuple(float(num) for num in value.replace('[', '').replace(']', '').split(', '))
    return float(value)

# conversion VTime a float
def time_to_years(time):
    h, m, s, ms, r = time.split(':')
    return h

df_converters = {
    VALUE_COL: parse_value,
    TIME_COL: time_to_years
}

salida = pd.read_csv(OUT_FILENAME,
                     delimiter=r'(?<!,)\s+',
                     engine='python',  # C engine doesnt work for regex
                     converters=df_converters,
                     names=[TIME_COL, PORT_COL, VALUE_COL]
                    )

# veamos cómo luce el dataframe con los datos de salida
salida.head() # filtro por índice

In [ ]:
def plot_product_demand(product):
    df = salida[[f'demand_p{product}' == port for port in salida[PORT_COL]]]
    xs = df[TIME_COL]
    ys = df[VALUE_COL]

    plt.figure(figsize=(10,5))
    plt.plot(xs, ys, marker='o')
    plt.grid(True)
    plt.xlabel('Tiempo [años]')
    plt.ylabel('Demanda [$]')
    plt.title(f'Demanda del producto {product} en el tiempo')

In [ ]:
plot_product_demand(0)